In [3]:
# Sensual content project 
# Importing necessary library
from PIL import Image
import requests
import os
import cv2
import pandas as pd
import numpy as np
import pytesseract

In [4]:
# Load the CSV file with image URLs using pandas
df = pd.read_csv('image2.csv')

In [5]:
# Load the reference "18+ banner" image
reference_banner_path = r'banner.jpg'  # Change to your reference image's path
reference_banner = cv2.imread(reference_banner_path)
reference_hsv = cv2.cvtColor(reference_banner, cv2.COLOR_BGR2HSV)

In [6]:
from PIL import Image
import pytesseract
import os

# # Specify the full path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Rijuma\Tesseract-OCR\tesseract.exe'

# Load the image
image_path = 'banner.jpg'  
image = Image.open(image_path)

# Extracting text from the image
text = pytesseract.image_to_string(image)

# Printing the extracted text
print(text)

TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [8]:
from PIL import Image
import pytesseract
import requests
import pandas as pd

# Specify the full path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Rijuma\Tesseract-OCR\tesseract.exe'

# Load the image
reference_image_path = 'banner.jpg'  # Replace with the path to your image
reference_image = Image.open(reference_image_path)

# Extract text from the image
reference_text = pytesseract.image_to_string(reference_image)

# Read the CSV file with image URLs using pandas
csv_file_path = r'C:\Users\Rijuma Chitrakar\Desktop\project\image.csv'
df = pd.read_csv(csv_file_path)

# Adding a new column 'Text Match Flag' to the DataFrame
df['Text Match Flag'] = 0  # Initialize with 0

for index, row in df.iterrows():
    image_url = row['url_main_image']
    
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        
        # Read the image using PIL
        img = Image.open(response.raw)
        
        # Extract text from the current image
        image_text = pytesseract.image_to_string(img)
        
        # Compare the extracted text with the reference text
        if reference_text in image_text:
            df.at[index, 'Text Match Flag'] = 1
        
        response.close()
    except requests.RequestException as e:
        print(f"Error fetching image from URL {image_url}: {e}")
    except Exception as e:
        print(f"An error occurred for URL {image_url}: {e}")

# Save the updated DataFrame to a new CSV file
output_csv_file_path = r'C:\Users\Rijuma Chitrakar\Desktop\project\output_text_flags.csv'
df.to_csv(output_csv_file_path, index=False)

print("Text match flags have been added and saved to output_text_flags.csv")


Error fetching image from URL https://np-live.slatic.net/p/35c4ba5756c0eb0ecedf1afda5d91c0b.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/35c4ba5756c0eb0ecedf1afda5d91c0b.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CEF9C90A10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/cd8c090ae18fd96ea88cb9d43c778ac5.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/cd8c090ae18fd96ea88cb9d43c778ac5.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CEF9E823D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/ff79d3a7ab82ae454f9335fa59060a53.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/ff79d3a7ab82ae4

In [13]:
from PIL import Image
import pytesseract
import requests
import pandas as pd

# Specify the full path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Rijuma\Tesseract-OCR\tesseract.exe'

# Load the reference banner image
reference_image_path = 'banner.jpg' 
reference_image = Image.open(reference_image_path)

# Extract text from the reference banner image
reference_text = pytesseract.image_to_string(reference_image)

# Read the CSV file with image URLs using pandas
df = pd.read_csv('image2.csv')

# Create a new column 'Text Match Flag' to the DataFrame
df['Text Match Flag'] = 0  # Initialize with 0

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    image_url = row['url_main_image']
    
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        
        # Read the image using PIL
        img = Image.open(response.raw)
        
        # Extract text from the current image
        image_text = pytesseract.image_to_string(img)
        
        # Compare the extracted text with the reference text
        if reference_text in image_text:
            df.at[index, 'Text Match Flag'] = 1
        
        response.close()
    except requests.RequestException as e:
        print(f"Error fetching image from URL {image_url}: {e}")
    except Exception as e:
        print(f"An error occurred for URL {image_url}: {e}")

# Save the updated DataFrame to a new CSV file
output_csv_file_path = r'C:\Users\Rijuma Chitrakar\Desktop\project\outputs.csv'
df.to_csv(output_csv_file_path, index=False)

print("Text match flags have been added and saved to output_text_flags.csv")


Error fetching image from URL https://static-01.daraz.com.np/p/ad0b87ce4ef3353ec258fdfec8426b45.png: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Error fetching image from URL https://np-live.slatic.net/p/fa7dd5dc29a67e33bc3242b1a715da12.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/fa7dd5dc29a67e33bc3242b1a715da12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CEF8B4AE90>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/fa7dd5dc29a67e33bc3242b1a715da12.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/fa7dd5dc29a67e33bc3242b1a715da12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CEF8B49950>: Failed to establish a new connection: [E

In [17]:
#extraxting product id and name

from PIL import Image
import pytesseract
import requests
import pandas as pd
from fuzzywuzzy import fuzz
from concurrent.futures import ThreadPoolExecutor

# Specify the full path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Rijuma\Tesseract-OCR\tesseract.exe'

# Load the reference banner image
reference_image_path = 'banner.jpg'  # Replace with the path to your image
reference_image = Image.open(reference_image_path)

# Extract text from the reference banner image
reference_text = pytesseract.image_to_string(reference_image)

# Read the CSV file with image URLs using pandas
csv_file_path = r'C:\Users\Rijuma Chitrakar\Desktop\project\image2.csv'
df = pd.read_csv(csv_file_path)

# Set a similarity threshold to determine if it's a match
similarity_threshold = 80

# Function to process a single row and return the product info if it does not match
def process_row(row):
    image_url = row.url_main_image
    
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        
        # Read the image using PIL
        img = Image.open(response.raw)
        
        # Extract text from the current image
        image_text = pytesseract.image_to_string(img)
        
        # Calculate text similarity using fuzz.ratio()
        similarity_score = fuzz.ratio(reference_text.lower(), image_text.lower())
        
        # Compare the similarity score with the threshold
        if similarity_score < similarity_threshold:
            return {
                'product_id': row.product_id,
                'product_name': row.product_name
            }
        
        response.close()
    except requests.RequestException as e:
        print(f"Error fetching image from URL {image_url}: {e}")
    except Exception as e:
        print(f"An error occurred for URL {image_url}: {e}")
    
    return None

# Process rows concurrently using ThreadPoolExecutor
non_matching_data = []
with ThreadPoolExecutor(max_workers=8) as executor:
    results = executor.map(process_row, df.itertuples(index=False))
    for data in results:
        if data:
            non_matching_data.append(data)

# Create a DataFrame for the non-matching data
output_df = pd.DataFrame(non_matching_data)

# Save the non-matching data to a CSV file
output_csv_file_path = r'C:\Users\Rijuma Chitrakar\Desktop\project\non_matching_data.csv'
output_df.to_csv(output_csv_file_path, index=False)

print(f"Non-matching data has been saved to {output_csv_file_path}")


Error fetching image from URL https://np-live.slatic.net/p/fa7dd5dc29a67e33bc3242b1a715da12.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/fa7dd5dc29a67e33bc3242b1a715da12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CE80F13350>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/fa7dd5dc29a67e33bc3242b1a715da12.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/fa7dd5dc29a67e33bc3242b1a715da12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CE802AC710>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/35c4ba5756c0eb0ecedf1afda5d91c0b.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/35c4ba5756c0eb0

In [16]:
#accurate test
from PIL import Image
import pytesseract
import requests
import pandas as pd
from fuzzywuzzy import fuzz

# Specify the full path to the tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Rijuma\Tesseract-OCR\tesseract.exe'

# Load the reference banner image
reference_image_path = 'banner.jpg'
reference_image = Image.open(reference_image_path)

# Extract text from the reference banner image
reference_text = pytesseract.image_to_string(reference_image)

# Read the CSV file with image URLs using pandas
df = pd.read_csv('image2.csv')

# Create a new column 'Text Match Flag' to the DataFrame
df['Text Match Flag'] = 0  # Initialize with 0

# Set a similarity threshold for fuzzy matching
similarity_threshold = 80

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    image_url = row['url_main_image']
    
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status()
        
        # Read the image using PIL
        img = Image.open(response.raw)
        
        # Extract text from the current image
        image_text = pytesseract.image_to_string(img)
        
        # Calculate text similarity using fuzz.ratio()
        similarity_score = fuzz.ratio(reference_text.lower(), image_text.lower())
        
        # Compare the similarity score with the threshold
        if similarity_score >= similarity_threshold:
            df.at[index, 'Text Match Flag'] = 1
        
        response.close()
    except requests.RequestException as e:
        print(f"Error fetching image from URL {image_url}: {e}")
    except Exception as e:
        print(f"An error occurred for URL {image_url}: {e}")

# Save the updated DataFrame to a new CSV file
output_csv_file_path = r'C:\Users\Rijuma Chitrakar\Desktop\project\outputs.csv'
df.to_csv(output_csv_file_path, index=False)

print("Text match flags have been added and saved to outputs.csv")


Error fetching image from URL https://np-live.slatic.net/p/fa7dd5dc29a67e33bc3242b1a715da12.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/fa7dd5dc29a67e33bc3242b1a715da12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CE81256850>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/fa7dd5dc29a67e33bc3242b1a715da12.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/fa7dd5dc29a67e33bc3242b1a715da12.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001CE8126E110>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Error fetching image from URL https://np-live.slatic.net/p/35c4ba5756c0eb0ecedf1afda5d91c0b.jpg: HTTPSConnectionPool(host='np-live.slatic.net', port=443): Max retries exceeded with url: /p/35c4ba5756c0eb0